このプログラムは、AMR-W/C法を実装し、その推定結果をCSVファイル形式で保存するためのものです。

In [2]:
import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")
Pkg.add("DataStructures")

   Resolving package versions...
    Updating `/workspaces/inulab_julia_devcontainer/Project.toml`
  [864edb3b] + DataStructures v0.18.22
  No Changes to `/workspaces/inulab_julia_devcontainer/Manifest.toml`


In [4]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
using DataStructures

# 共通ライブラリの読み込み
include("./libs/display-latex.jl")
include("./libs/crisp-pcm.jl")
include("./libs/analysis-indicators.jl")
include("./libs/solve-deterministic-ahp.jl")

# ★★★ W/C法を実装したライブラリを読み込む ★★★
include("./libs/MMR-W.jl");

In [5]:
# 評価基準数
num_criteria = [4,5,6,7,8];
# 真の区間重要度のセッティング
true_importance = ["A", "B", "C", "D", "E"];

In [8]:
# 真の区間重要度を読み込み
for N in num_criteria
    for setting in true_importance

        # --- 結果を格納するDataFrameの準備 --- #
        Simp_columns = OrderedDict()
        push!(Simp_columns, "Num" => Int[])
        for i in 1:N
            push!(Simp_columns, "wL[$(i-1)]" => Float64[])
            push!(Simp_columns, "wR[$(i-1)]" => Float64[])
        end
        push!(Simp_columns, "Sum_of_Width" => Float64[])
        Simp = DataFrame(Simp_columns)

        F_columns = OrderedDict()
        push!(F_columns, "Num" => Int[])
        for i in 1:N
            push!(F_columns, "F_$i" => Float64[])
        end
        push!(F_columns, "Average" => Float64[])
        index_Fi = DataFrame(F_columns)

        P_columns = OrderedDict()
        push!(P_columns, "Num" => Int[])
        for i in 1:N
            push!(P_columns, "P_$i" => Float64[])
        end
        push!(P_columns, "Average" => Float64[])
        index_Pi = DataFrame(P_columns)

        Q_columns = OrderedDict()
        push!(Q_columns, "Num" => Int[])
        for i in 1:N
            push!(Q_columns, "Q_$i" => Float64[])
        end
        push!(Q_columns, "Average" => Float64[])
        index_Qi = DataFrame(Q_columns)

        R_columns = OrderedDict()
        push!(R_columns, "Num" => Int[])
        for i in 1:N
            push!(R_columns, "R_$i" => Float64[])
        end
        push!(R_columns, "Average" => Float64[])
        index_Ri = DataFrame(R_columns)

        T_E_matome_all_columns = OrderedDict()
        push!(T_E_matome_all_columns, "Index" => String[])
        push!(T_E_matome_all_columns, "Mean" => Float64[])
        push!(T_E_matome_all_columns, "Var" => Float64[])
        push!(T_E_matome_all_columns, "SD" => Float64[])
        T_E_matome_all = DataFrame(T_E_matome_all_columns)

        # --- データ読み込み --- #
        df = CSV.File("./data/N"*string(N)*"_"*setting*"_Given_interval_weight.csv", header=false) |> DataFrame
        T = Vector{Interval{Float64}}()
        for i in 1:N
            push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
        end

        df1 = CSV.File("./data/N"*string(N)*"_a3_"*setting*"_PCM_int.csv", header=false) |> DataFrame
        subdfs = split_dataframe(df1, N);
        
        # --- 1000個のPCMでシミュレーション --- #
        # P = Matrix{Float64}(undef, N, 1000)
        # Q = Matrix{Float64}(undef, N, 1000)
        # R = Matrix{Float64}(undef, N, 1000)
        # F = Matrix{Float64}(undef, N, 1000)

        for (i, subdf) in enumerate(subdfs)
            A = Matrix(subdf)

            # ★★★ W/C法で区間重要度を推定 ★★★
            solution = MMR_WC(A, EV)
            E = solution.W

            # Simp.csv用のデータ整形
            E_data = Vector{Float64}()
            for j in 1:N
                push!(E_data, inf(E[j]))
                push!(E_data, sup(E[j]))
            end
            push!(Simp, (i, E_data..., sum(diam.(E)))) # `c_diam`から`diam`に変更（IntervalArithmetic.jl v1.0以降）

            # 評価指標の計算
            # P[:, i] = calculate_P(T, E)
            # Q[:, i] = calculate_Q(T, E)
            # R[:, i] = calculate_R(T, E)
            # F[:, i] = calculate_F(T, E)

            # # 各指標ファイル用のデータ整形
            # Pi_data = [P[j, i] for j in 1:N]
            # push!(index_Pi, (i, Pi_data..., mean(P[:,i])))

            # Qi_data = [Q[j, i] for j in 1:N]
            # push!(index_Qi, (i, Qi_data..., mean(Q[:,i])))

            # Ri_data = [R[j, i] for j in 1:N]
            # push!(index_Ri, (i, Ri_data..., mean(R[:,i])))

            # Fi_data = [F[j, i] for j in 1:N]
            # push!(index_Fi, (i, Fi_data..., mean(F[:,i])))
        end

        println("N = ", N, ", setting = ", setting, " finished.")

        # --- 平均・分散の計算 --- #
        # P_ave = mean(P); Q_ave = mean(Q); R_ave = mean(R); F_ave = mean(F)
        # P_var = var(P); Q_var = var(Q); R_var = var(R); F_var = var(F)

        # push!(T_E_matome_all, ("P", P_ave, P_var, sqrt(P_var)))
        # push!(T_E_matome_all, ("Q", Q_ave, Q_var, sqrt(Q_var)))
        # push!(T_E_matome_all, ("R", R_ave, R_var, sqrt(R_var)))
        # push!(T_E_matome_all, ("F", F_ave, F_var, sqrt(F_var)))

        # --- CSVファイルへの書き出し --- #
        # ★★★ 出力フォルダ名を変更 ★★★
        output_folder_name = "MMRwc"

        root_folder = "./out/Simp/N=" * string(N) * "/a3/" * setting
        output_path = root_folder * "/" * output_folder_name
        
        if !isdir(output_path)
            mkpath(output_path) # 入れ子のディレクトリもまとめて作成
        end

        Simp_output_path = output_path * "/Simp.csv"
        # index_Fi_output_path = output_path * "/index_Fi.csv"
        # index_Pi_output_path = output_path * "/index_Pi.csv"
        # index_Qi_output_path = output_path * "/index_Qi.csv"
        # index_Ri_output_path = output_path * "/index_Ri.csv"
        # T_E_matome_all_output_path = output_path * "/T_E_matome_all.csv"

        CSV.write(Simp_output_path, Simp)
        # CSV.write(index_Fi_output_path, index_Fi)
        # CSV.write(index_Pi_output_path, index_Pi)
        # CSV.write(index_Qi_output_path, index_Qi)
        # CSV.write(index_Ri_output_path, index_Ri)
        # CSV.write(T_E_matome_all_output_path, T_E_matome_all)
    end
end

N = 4, setting = A finished.
N = 4, setting = B finished.
N = 4, setting = C finished.
N = 4, setting = D finished.
N = 4, setting = E finished.
N = 5, setting = A finished.
N = 5, setting = B finished.
N = 5, setting = C finished.
N = 5, setting = D finished.
N = 5, setting = E finished.
N = 6, setting = A finished.
N = 6, setting = B finished.
N = 6, setting = C finished.
N = 6, setting = D finished.
N = 6, setting = E finished.
N = 7, setting = A finished.
N = 7, setting = B finished.
N = 7, setting = C finished.
N = 7, setting = D finished.
N = 7, setting = E finished.
N = 8, setting = A finished.
N = 8, setting = B finished.
N = 8, setting = C finished.
N = 8, setting = D finished.
N = 8, setting = E finished.
